In [ ]:
import gzip
import os
from warcio.archiveiterator import ArchiveIterator
import tldextract
import matplotlib.pyplot as plt

# Diretório onde estão os arquivos WARC
warc_dir = '../data'
base_name = 'corpus_{:03d}.warc.gz'

all_urls = []
main_domains = []

# Loop pelos arquivos corpus_000 até corpus_100
for i in range(101):
    file_name = base_name.format(i)
    file_path = os.path.join(warc_dir, file_name)
    if not os.path.exists(file_path):
        continue

    try:
        with gzip.open(file_path, 'rb') as stream:
            for record in ArchiveIterator(stream):
                url = record.rec_headers.get_header('WARC-Target-URI')
                if url:
                    all_urls.append(url)
                    # Extrai domínio principal sem subdomínio
                    extracted = tldextract.extract(url)
                    domain = f"{extracted.domain}.{extracted.suffix}"
                    main_domains.append(domain)
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")
        continue

# Dados para gráfico
num_urls = len(all_urls)
num_unique_domains = len(set(main_domains))

# Gráfico
plt.figure(figsize=(8, 6))
plt.bar(['Total de URLs', 'Domínios Principais Únicos'], [num_urls, num_unique_domains], color=['blue', 'green'])
plt.title('Quantidade de URLs e Domínios Principais')
plt.ylabel('Quantidade')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
set(main_domains)